# Part I. Prepare data

### 1. User $\times$ Item $\times$ Month (from 1 to 12)

In [ ]:
# construct tensor I here

### 2. User $\times$ Item $\times$ Rating (from 1 to 10)

In [ ]:
# contruct tensor II here

# Part II. Decompose (more decompositions?)

### 1.[ Tucker decomposition](http://tensorly.org/stable/user_guide/tensor_decomposition.html)

In [ ]:
# decompose both tensors

### 2. [HOSVD decomposition](https://pytorch.org/docs/stable/generated/torch.svd.html) or [Lowrank HOSVD decomposition](https://pytorch.org/docs/stable/generated/torch.svd_lowrank.html)

In [ ]:
# decompose both tensors

### 3. [Tensor train decomposition ](https://github.com/oseledets/TT-Toolbox)

In [ ]:
# decompose both tensors

### Bonus: Use [Polara](https://github.com/evfro/polara)

In [ ]:
# use Polara

# Part III. Recommend (no clue how to do it yet)

In [ ]:
# time_metric
# quality_metric

In [ ]:
# Tensor 1: Tucker result metrics

# Tensor 1: HOSVD result metrics

# Tensor 1: Tensor train result metrics

In [ ]:
# Tensor 2: Tucker result metrics

# Tensor 2: HOSVD result metrics

# Tensor 2: Tensor train result metrics

# Part IV. Visuals

In [ ]:
# performance plot: Tucker vs HOSVD vs Tensor Train

In [ ]:
# quality plot: "Tensor 1: HOSVD" vs "Tensor 1: Tucker" vs "Tensor 1: TT" vs "Tensor 2: HOSVD" vs "Tensor 2: Tucker" vs "Tensor 2: TT"